# Locations with Most Decline Due to Lockdown

####  Import packages and start the session

In [1]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 2
%extra_py_files s3://pedestrian-analysis-working-bucket/glue-job-scripts/util.py

import sys, io, util
from datetime import datetime, timedelta

from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
)

from pyspark.context import SparkContext
from pyspark.sql.functions import sum, col, rank, desc, lit, when
from pyspark.sql.window import Window
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Current idle_timeout is 2800 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 3.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 2
Extra py files to be included:
s3://pedestrian-analysis-working-bucket/glue-job-scripts/util.py
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::632753217422:role/pedestrians-analysis-notebook-role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: dff3e5fe-335e-444b-b215-d4446a7aa0f7
Job Type: glueetl
Applying the following

####  Create output glue table if it doesn't already exist
##### The results of this notebook will be loaded into this table

In [2]:
BUCKET_NAME = 'pedestrian-analysis-working-bucket'
DATABASE_NAME = 'pedestrian_analysis_report'
OUTPUT_TABLE_NAME = 'report_location_declines_due_to_lockdown'

schema = StructType([
    StructField("location_name", StringType(), True),
    StructField("2019_count", IntegerType(), True),
    StructField("2022_count", IntegerType(), True),
    StructField("decline", IntegerType(), True),
    StructField("decline_percent", DoubleType(), True),
])

s3_path = f"s3://{BUCKET_NAME}/report/{OUTPUT_TABLE_NAME}/"
util.create_glue_catalog_table(DATABASE_NAME, OUTPUT_TABLE_NAME, schema, s3_path)

Table pedestrian_analysis_report.report_location_declines_due_to_lockdown not found in the Glue Data Catalog. Creating table...
Table pedestrian_analysis_report.report_location_declines_due_to_lockdown created in the Glue Data Catalog


####  Load sensor_counts_by_day

In [3]:
sensor_counts_df = glueContext.create_dynamic_frame.from_catalog(
    database="pedestrian_analysis_raw",
    table_name="sensor_counts"
).toDF()

sensor_counts_df.show(10, truncate=False)

+-------+-------------------+---------+--------------------------------------+------------+
|id     |date_time          |sensor_id|sensor_name                           |hourly_count|
+-------+-------------------+---------+--------------------------------------+------------+
|2902119|2019-11-12T22:00:00|11       |Waterfront City                       |27          |
|763567 |2014-01-24T10:00:00|25       |Melbourne Convention Exhibition Centre|436         |
|1028480|2015-01-06T16:00:00|23       |Spencer St-Collins St (South)         |760         |
|331173 |2011-07-08T12:00:00|10       |Victoria Point                        |569         |
|430567 |2012-03-03T19:00:00|1        |Bourke Street Mall (North)            |847         |
|631323 |2013-06-14T14:00:00|4        |Town Hall (West)                      |3219        |
|573323 |2013-01-30T13:00:00|16       |Australia on Collins                  |2931        |
|408282 |2012-01-12T04:00:00|18       |Collins Place (North)                 |1 

####  Load sensor_reference_data

In [4]:
sensor_reference_df = glueContext.create_dynamic_frame.from_catalog(
    database="pedestrian_analysis_raw",
    table_name="sensor_reference_data"
).toDF()

sensor_reference_df.show(10)

+-----------+-----------+-----------------+------------+--------------------+-----------+-------------+------------+--------------------+--------------------+-----------+------+
|direction_1|direction_2|installation_date|    latitude|            location|location_id|location_type|   longitude|                note|  sensor_description|sensor_name|status|
+-----------+-----------+-----------------+------------+--------------------+-----------+-------------+------------+--------------------+--------------------+-----------+------+
|      North|      South|       2009-03-25|-37.81101524|{lon -> 144.96429...|          3|      Outdoor|144.96429485|                null|   Melbourne Central|   Swa295_T|     A|
|      North|      South|       2009-03-23|-37.81487988|{lon -> 144.96608...|          4|      Outdoor| 144.9660878|                null|    Town Hall (West)|   Swa123_T|     A|
|       East|       West|       2021-12-21|-37.79432415|{lon -> 144.92973...|         85|      Outdoor|144.929

#### Calculate sensor counts 2019
##### Because the cutsoff at 2022-11-01, for the purpose of this analysis we are setting the end of each year to November 11

In [5]:
sensor_counts_2019_df = sensor_counts_df.filter(
    col('date_time') >= '2018-11-01'
).filter(
    col('date_time') < '2019-11-01'
).groupBy(
    'sensor_id'
).agg(
    sum('hourly_count').alias('count_2019')
)

sensor_counts_2019_df.show(10)

+---------+----------+
|sensor_id|count_2019|
+---------+----------+
|       37|   1253474|
|       46|    864532|
|       12|   2069570|
|       52|   3642230|
|       18|   3058886|
|       50|   2340233|
|       56|   2806452|
|       14|   4280452|
|       25|   4433454|
|       62|    126595|
+---------+----------+
only showing top 10 rows


#### Calculate sensor counts 2022

In [6]:
sensor_counts_2022_df = sensor_counts_df.filter(
    col('date_time') >= '2021-11-01'
).filter(
    col('date_time') < '2022-11-01'
).groupBy(
    'sensor_id'
).agg(
    sum('hourly_count').alias('count_2022')
)

sensor_counts_2022_df.show(10)

+---------+----------+
|sensor_id|count_2022|
+---------+----------+
|       12|   1800608|
|       65|   3066693|
|       67|   2746840|
|       46|    524097|
|       18|    743849|
|       25|   6695435|
|       70|   1343628|
|       14|   2486303|
|       37|    971716|
|       56|   2114456|
+---------+----------+
only showing top 10 rows


#### Calculate the decline and decline percentages for each sensor

In [7]:
sensor_decline_df = sensor_counts_2019_df.join(
    sensor_counts_2022_df,
    on='sensor_id',
    how='inner'
).withColumn(
    'decline',
    (col('count_2019') - col('count_2022'))
).withColumn(
    'decline_percent',
    ((col('count_2019') - col('count_2022')) / col('count_2019')) * 100
)

sensor_decline_df.show(10)

+---------+----------+----------+--------+------------------+
|sensor_id|count_2019|count_2022| decline|   decline_percent|
+---------+----------+----------+--------+------------------+
|       34|   1268286|     20796| 1247490| 98.36030674469323|
|       22|  12975568|    702581|12272987| 94.58535456790793|
|       15|   7255167|    456515| 6798652| 93.70772581802734|
|        7|   2257446|    394437| 1863009| 82.52728968932146|
|       18|   3058886|    743849| 2315037| 75.68235625649338|
|       10|   1631873|    475750| 1156123| 70.84638326634487|
|       52|   3642230|   1063063| 2579167| 70.81285366382684|
|        9|   6833188|   2064815| 4768373| 69.78255244843257|
|       57|   6861050|   2414386| 4446664| 64.81025499012542|
|       43|   1630186|    610936| 1019250|62.523540258596256|
+---------+----------+----------+--------+------------------+
only showing top 10 rows


In [6]:
#### Join reference and select relevant columns

In [14]:
sensor_decline_df = sensor_decline_df.join(
    sensor_reference_df,
    col("sensor_id") == col("location_id"),
    "left"
)

sensor_decline_df = sensor_decline_df.select(
    col('sensor_id'),
    col('sensor_description').alias('location_name'),
    col('count_2019').cast('int'),
    col('count_2022').cast('int'),
    col('decline'),
    col('decline_percent')
).orderBy(
    desc('decline_percent')
)

sensor_decline_df.show(100, truncate=False)

+---------+--------------------------------------+---+---+---------+-------------------+
|sensor_id|location_name                         |1  |2  |decline  |decline_percent    |
+---------+--------------------------------------+---+---+---------+-------------------+
|34       |null                                  |1  |2  |1247490  |98.36030674469323  |
|22       |null                                  |1  |2  |12272987 |94.58535456790793  |
|15       |null                                  |1  |2  |6798652  |93.70772581802734  |
|7        |Birrarung Marr                        |1  |2  |1863009  |82.52728968932146  |
|18       |Collins Place (North)                 |1  |2  |2315037  |75.68235625649338  |
|10       |Victoria Point                        |1  |2  |1156123  |70.84638326634487  |
|52       |Elizabeth St-Lonsdale St (South)      |1  |2  |2579167  |70.81285366382684  |
|9        |Southern Cross Station                |1  |2  |4768373  |69.78255244843257  |
|57       |Bourke St 

In [15]:
util.upload_to_s3(glueContext, sensor_decline_df, s3_path)

Successfully Uploaded to s3 in path: s3://pedestrian-analysis-working-bucket/report/report_location_declines_due_to_lockdown/
